# AI-Hub SharePoint Agent

To create an AI agent that is capable to generate pages in SharePoint, you need to first connect to an LLM, in this case, OpenAI GPT-5 will be used

## Testing Connection to OpenAI API

OpenAI API needs an API-KEY to work. This is stored in an Azure vault-key

In [14]:
from openai import OpenAI
import re
import json
import sys
sys.path.append("..")

from agent.credentials import get_agent_config

In [ ]:
# get_agent_config returns a 'Singleton', which means that it will always return the same instance, so whenever
# you call it, you will get the same configuration object. Avoiding multiple vault calls and ensuring consistency.
agent_config = get_agent_config()

Connecting to the OpenAI API

*Code adapted, source [here](https://github.com/openai/openai-python)*

In [10]:
openai_client = OpenAI(api_key=agent_config.OPENAI_KEY.get_secret_value())

The main objective of the LLM is to generate a JSON containing the structure of a SharePoint page based on a specific topic, which will be a parameter

In [8]:
topic = "What is an LLM and how it works?"

In [12]:
response = openai_client.responses.create(
    model="gpt-4o",
    instructions="""
    You are a content generation agent specialized in designing modern SharePoint pages for public-facing communication. Your expertise lies in SharePoint layout design and in simplifying complex topics related to artificial intelligence for a general audience.

    Your role is to act as a science communicator and visual designer. You will generate the structure and content of a SharePoint page that introduces and explains the topic provided by the user.

    The SharePoint page must be:
    - Visually engaging
    - Easy to understand for non-technical readers
    - Structured for clarity and flow

    Each page must include:
    - A compelling and accessible title
    - Multiple sections with clear headers and explanatory content
    - Visual resources (e.g., images, diagrams, external links)
    - A final summary that reinforces key ideas
    - A link to a YouTube video that helps explain the topic further

    Output format: JSON with the following fields:
    - "title": string
    - "sections": array of objects with "header" and "content"
    - "visuals": array of strings (URLs or descriptions)
    - "summary": string

    You will receive a single input:  
    **{{topic}}** — the AI-related concept to be explained (e.g., Machine Learning, NLP, Neural Networks, AI Ethics, Azure AI Services, GCP Vertex AI, BigQuery ML).
        """,
    input = "Neural Networks",
    temperature=0.7

)

In [13]:
response.output_text

'```json\n{\n  "title": "Understanding Neural Networks: The Brain Behind AI",\n  "sections": [\n    {\n      "header": "Introduction to Neural Networks",\n      "content": "Neural networks are a key technology behind artificial intelligence, inspired by the human brain. They are made up of interconnected nodes, similar to neurons, that work together to process information."\n    },\n    {\n      "header": "How Neural Networks Work",\n      "content": "Neural networks consist of layers: input, hidden, and output layers. Each node in a layer connects to nodes in the next layer. Data enters through the input layer, gets processed in hidden layers, and results are outputted. This process allows for complex data analysis and decision-making."\n    },\n    {\n      "header": "Applications of Neural Networks",\n      "content": "From image and speech recognition to autonomous vehicles and medical diagnostics, neural networks power many modern technologies. They are capable of learning and imp

In [ ]:
def parse_openai_json(text: str) -> dict:
    """
    Parses the text generated by OpenAI and returns a dict with the JSON.
    Removes code block delimiters and unnecessary spaces.

    Args:
        text (str): The text generated by OpenAI, possibly containing JSON inside code block delimiters.

    Returns:
        dict: The parsed JSON as a Python dictionary.
"""
    # Elimina delimitadores de bloque de código (```json ... ```)
    match = re.search(r"```json\s*(\{.*\})\s*```", text, re.DOTALL)
    if match:
        json_str = match.group(1)
    else:
        # Si no hay delimitadores, intenta buscar el primer objeto JSON
        match = re.search(r"(\{.*\})", text, re.DOTALL)
        if match:
            json_str = match.group(1)
        else:
            raise ValueError("No se encontró un bloque JSON válido en el texto.")

    return json.loads(json_str)

In [17]:
parsed_json = parse_openai_json(response.output_text)

In [18]:
parsed_json

{'title': 'Understanding Neural Networks: The Brain Behind AI',
 'sections': [{'header': 'Introduction to Neural Networks',
   'content': 'Neural networks are a key technology behind artificial intelligence, inspired by the human brain. They are made up of interconnected nodes, similar to neurons, that work together to process information.'},
  {'header': 'How Neural Networks Work',
   'content': 'Neural networks consist of layers: input, hidden, and output layers. Each node in a layer connects to nodes in the next layer. Data enters through the input layer, gets processed in hidden layers, and results are outputted. This process allows for complex data analysis and decision-making.'},
  {'header': 'Applications of Neural Networks',
   'content': 'From image and speech recognition to autonomous vehicles and medical diagnostics, neural networks power many modern technologies. They are capable of learning and improving over time, making them essential in various fields.'},
  {'header': '